In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
from Preprocessing import MatchAnalyzer as ma
from Database import FootballDB

In [3]:
pd.options.display.max_columns=300

### 1.1 데이터 경로 설정

In [4]:
cur_path = os.getcwd()
main_path = os.path.dirname(cur_path)

In [5]:
source_path = os.path.join(main_path, '1. Data')
prm_source_path = os.path.join(source_path, '1. Premier League')

### 1.2 데이터 불러오기

In [6]:
season_stat = {}
seasons = (season for season in os.listdir(prm_source_path) if season!='info')
for season in seasons:
    if season not in season_stat:
        season_stat[season] = {}    
    season_stat_path = os.path.join(prm_source_path, season)
    for stat in os.listdir(season_stat_path):
        stat_nm = os.path.splitext(stat)[0]
        file_path = os.path.join(season_stat_path, stat)
        season_stat[season][stat_nm] = file_path

### 2. 데이터 전처리 및 DB Insert

In [7]:
db = FootballDB('localhost', 'postgres', 5432, 'alsdud12', 'football')

In [8]:
league = 'Premier League'
for season, datas in season_stat.items():
    print(season)
    match_data = pd.read_csv(datas['match'])
    match_data.dropna(subset=['HomeTeam', 'AwayTeam'], how='any', axis=0, inplace=True) # 팀이름 결측치 행 제거
    match_data['HomeTeam'] = match_data['HomeTeam'].apply(lambda x: x.replace("'", "") if x.__contains__("'") else x)
    match_data['AwayTeam'] = match_data['AwayTeam'].apply(lambda x: x.replace("'", "") if x.__contains__("'") else x)
    
    dt_season_position = ma.season_position(league, season, match_data)
    dt_season = ma.season(dt_season_position)
    dt_season_match = ma.season_match_info(league, season, match_data)
    dt_teams = dt_season[['team']]
    
    db.insertTable('tbl_season_position', dt_season_position)
    db.insertTable('tbl_season', dt_season)
    db.insertTable('tbl_season_match', dt_season_match)
    db.insertTable('tbl_team', dt_teams, True, 'team')

14-15
15-16
16-17
17-18
18-19
19-20
20-21
21-22
22-23
23-24


In [13]:
db.dropDB()
db.createDB()
db.createTables()